In [1]:
import src.data.breathe_data as breathe_data
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots
import src.models.helpers as mh

import pandas as pd
import numpy as np

In [2]:
df = breathe_data.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx")

INFO:root:* Checking for same day measurements *


In [3]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx ecFEF25-75 % ecFEV1 (%),idx O2 saturation (%)
0,101,2019-01-25,1.31,97,0.54,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492,41.221374,26,20,47
1,101,2019-01-26,1.31,98,0.57,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827,43.511450,26,21,48
2,101,2019-01-27,1.31,96,0.67,1.31,0.69,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157,51.145038,26,25,46
3,101,2019-01-28,1.30,96,0.69,1.31,0.69,Male,173.0,53,3.610061,97.150104,36.287474,36.010470,98.816157,53.076923,26,26,46
4,101,2019-01-29,1.28,98,0.60,1.30,0.69,Male,173.0,53,3.610061,97.150104,36.010470,35.456463,100.874827,46.875000,26,23,48


In [5]:
# Initialize variables (not used for inference)
# interconnected_AR = True
interconnected_AR = "fix message up to HFEV1 to truncated uniform"
ar_change_cpt_suffix = "_padded_identity_15"
ar_change_cpt_suffix = "_shift_span_[-20;20]_samples"
save = True
if interconnected_AR == "fix message up to HFEV1 to truncated uniform":
    inteconnected_AR_str = "interdays AR v2 with trunc uni message to HFEV1, "
else:
    inteconnected_AR_str = ""


def infer_and_plot_for_id(df_for_ID, diff_threshold=1e-8):
    df_for_ID = df_for_ID.reset_index(drop=True)
    print(f"\nID: {df_for_ID.ID.iloc[0]}")
    print(f"Number of datapoints: {len(df_for_ID)}")

    height = df_for_ID.Height.iloc[0]
    age = df_for_ID.Age.iloc[0]
    sex = df_for_ID.Sex.iloc[0]
    (
        _,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_long_model_shared_healthy_vars_and_temporal_ar(
        height, age, sex, ia_prior="breathe", ar_change_cpt_suffix=ar_change_cpt_suffix
    )

    key_hfev1 = f"['{ecFEV1.name}', '{HFEV1.name}', '{AR.name}'] -> {HFEV1.name}"
    key_ho2sat = f"['{O2SatFFA.name}', '{HO2Sat.name}', '{AR.name}'] -> {HO2Sat.name}"
    HFEV1.set_factor_node_key(key_hfev1)
    HO2Sat.set_factor_node_key(key_ho2sat)

    vars = [AR, IA]
    shared_vars = [HFEV1, HO2Sat]
    # obs_vars = [ecFEV1.name, O2Sat.name]
    # obs_vars = [ecFEV1.name, O2Sat.name, ecFEF2575prctecFEV1.name]
    obs_vars = [ecFEV1.name, ecFEF2575prctecFEV1.name]
    # obs_vars = [ecFEV1.name]

    df_query_res, df_res_before_convergence, shared_vars_final = (
        slicing.query_back_and_forth_across_days_AR(
            df_for_ID,
            inf_alg,
            shared_vars,
            vars,
            obs_vars,
            diff_threshold,
            debug=False,
            max_passes=1,
            interconnect_AR=interconnected_AR,
        )
    )

    colorscale = [
        [0, "white"],
        [0.01, "red"],
        [0.05, "yellow"],
        [0.1, "cyan"],
        [0.6, "blue"],
        [1, "black"],
    ]

    obs_vars_str = ", ".join([mh.name_to_abbr(var) for var in obs_vars])
    ar_prior_str = f"AR {ar_change_cpt_suffix}"
    passes = "back and forth"

    title = f"ID {df_for_ID.ID[0]} - obs. {obs_vars_str}, {passes}, {inteconnected_AR_str}{ar_prior_str} ({df_for_ID.Sex[0]}, {df_for_ID.Age[0]}yr, {df_for_ID.Height[0]}cm)"
    slicing.plot_posterior_validation(
        df_res_before_convergence,
        HFEV1,
        AR,
        df_for_ID,
        title,
        colorscale,
        save,
    )

    title = f"ID {df_for_ID.ID[0]} - obs. {obs_vars_str}, {inteconnected_AR_str}{ar_prior_str} ({df_for_ID.Sex[0]}, {df_for_ID.Age[0]}yr, {df_for_ID.Height[0]}cm)"
    slicing.plot_query_res(
        df_for_ID,
        ecFEV1,
        O2Sat,
        df_query_res,
        AR,
        IA,
        HFEV1,
        HO2Sat,
        title,
        colorscale,
        save,
    )
    return df_query_res, df_res_before_convergence, shared_vars_final


# interesting_ids = [
#     "132",
#     "146",
#     "177",
#     "180",
#     "202",
#     "527",
#     "117",
#     "131",
#     "134",
#     "191",
#     "139",
#     "253",
#     "101",
# ]

# Consecutive entries are less than 3 days apart until
# 101     (592, 1680)

df_for_ID = df[df["ID"] == "101"].iloc[:591]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "405"]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "272"].iloc[:417]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "201"].iloc[:289]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
df_for_ID = df[df["ID"] == "203"].iloc[:285]
df_query_res, df_res_before_convergence, shared_vars_final = infer_and_plot_for_id(
    df_for_ID, diff_threshold=1e-2
)
# df[df.ID.isin(interesting_ids)].groupby("ID").apply(
#     lambda df_for_ID: infer_and_plot_for_id(
#         df_for_ID, diff_threshold=1e-6
#     )
# )


ID: 101
Number of datapoints: 591
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 1.3749322619942235
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 1.4626076335121974
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 0.2407075082496745
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 4.034620778996647e-11

ID: 405
Number of datapoints: 1035
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 1.3780305949438498
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 1.48134211031637
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 0.2672534930694501
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 2.2661681250639776e-12

ID: 272
Number of datapoints: 417
Pass 1 - Posteriors' diff for Healthy FEV1 (L): 1.5427356338828766
Pass 1 - Posteriors' diff for Healthy O2 saturation (%): 1.4714769810253396
Pass 2 - Posteriors' diff for Healthy FEV1 (L): 0.11282209578192004
Pass 2 - Posteriors' diff for Healthy O2 saturation (%): 2.2545419848766977e-11

ID: 201
Number of datapoi

In [7]:
import src.data.helpers as dh

dftmp = dh.load_excel(
    f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_with_two_days_model_O2Sat_FEV1.xlsx",
    # f"{dh.get_path_to_main()}/ExcelFiles/BR/Refining_F3/infer_AR_with_two_days_model_O2Sat_ecFEV1.xlsx",
    [AR.name],
    ["Day"],
).drop(columns=["Unnamed: 0", HO2Sat.name, IA.name, HFEV1.name])

In [14]:
def get_consec_days(df):
    df = df.sort_values(by="Date Recorded")
    df["Prev day"] = df["Date Recorded"].shift(1)
    # Compute n days between measurements
    df["Days elapsed"] = df["Date Recorded"] - df["Prev day"]
    # Remove nan
    df = df.dropna(subset=["Days elapsed"])

    # Get first idx where Days elapsed is greater than 1
    idx = df[df["Days elapsed"] > pd.Timedelta(days=3)].index
    print(idx)

    return df


df_tmp = df[df.ID == "101"]
get_consec_days(df_tmp).iloc[:592]
# Not finished